In [1]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from email.mime.text import MIMEText
from email.MIMEMultipart import MIMEMultipart
from email.MIMEBase import MIMEBase
import mimetypes
import base64
import csv
# If modifying these scopes, delete the file token.pickle.
#SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
SCOPES = ['https://www.googleapis.com/auth/gmail.send']

In [2]:
def create_message_with_attachment(
    sender, to, subject, replyto, message_text, file):
  """Create a message for an email.

  Args:
    sender: Email address of the sender.
    to: Email address of the receiver.
    subject: The subject of the email message.
    message_text: The text of the email message.
    file: The path to the file to be attached.

  Returns:
    An object containing a base64url encoded email object.
  """
  message = MIMEMultipart()
  message['to'] = to
  message['from'] = sender
  message['subject'] = subject
  message['Reply-to'] = replyto

  msg = MIMEText(message_text, 'html') #message = MIMEText(message_text, 'html')
  message.attach(msg)

  content_type, encoding = mimetypes.guess_type(file)

  if content_type is None or encoding is not None:
    content_type = 'application/octet-stream'
  main_type, sub_type = content_type.split('/', 1)
  if main_type == 'text':
    fp = open(file, 'rb')
    msg = MIMEText(fp.read(), _subtype=sub_type)
    fp.close()
  elif main_type == 'image':
    fp = open(file, 'rb')
    msg = MIMEImage(fp.read(), _subtype=sub_type)
    fp.close()
  elif main_type == 'audio':
    fp = open(file, 'rb')
    msg = MIMEAudio(fp.read(), _subtype=sub_type)
    fp.close()
  else:
    fp = open(file, 'rb')
    msg = MIMEBase(main_type, sub_type)
    msg.set_payload(fp.read())
    fp.close()
  filename = os.path.basename(file)
  msg.add_header('Content-Disposition', 'attachment', filename=filename)
  message.attach(msg)

  return {'raw': base64.urlsafe_b64encode(message.as_string())}

In [3]:
def send_message(service, user_id, message):
  """Send an email message.

  Args:
    service: Authorized Gmail API service instance.
    user_id: User's email address. The special value "me"
    can be used to indicate the authenticated user.
    message: Message to be sent.

  Returns:
    Sent Message.
  """
  try:
    message = (service.users().messages().send(userId=user_id, body=message)
               .execute())
    print("Message Id: %s" % message['id'])
    return message
  except Exception, error:
    print('An error occurred: %s' % error)

In [4]:
def send():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)

    # Call the Gmail API
    #results = service.users().labels().list(userId='me').execute()
    #labels = results.get('labels', [])

    #if not labels:
    #    print('No labels found.')
    #else:
    #    print('Labels:')
    #    for label in labels:
    #        print(label['name'])
    send_message(service, 'me', message)

In [8]:
with open("testing_list.csv", "r") as f:
    reader = csv.reader(f, delimiter=",")
    for i, line in enumerate(reader):
        #print('line[{}] = {}'.format(i, line))
        #print(line[0].strip(","))
        userid = line[0]#.zfill(4)
        fn = "certs\\KUAR_certificate__Data Set "+userid+".pdf"
        name = line[1]
        email = line[2]
        replyto = "no-reply@ku.edu.tr"
        print(userid, name, email, fn)
        
        subject = 'KUAR Certificate'
        
        message_text = """<p style="font-weight: 400;">Merhaba """+ name +""",</p>
        <p style="font-weight: 400;">Yaptığımız inceleme sonucunda ekteki sertifikayı kazandığınızı bildirmekten mutluluk duyarız.</p>
        <p style="font-weight: 400;">Bundan sonra KUAR e-postasından gelen bildirimlere dikkat etmenizi rica ederiz.&nbsp;<strong>Size zaman zaman verilecek 10 g&ouml;revi&nbsp;</strong>yerine getirdiğinizde,&nbsp;<strong>deneyimli katılımcı sertifikasını</strong>&nbsp;da almaya hak kazanacaksınız.</p>
        <p style="font-weight: 400;">Deneyimli katılımcı sertifikasını alanlar proje imkanları doğrultusunda &ccedil;eşitli teşvik &ouml;d&uuml;lleri alabilir.</p>
        <p style="font-weight: 400;"><strong>Sizden ricamız Designlab instagram hesabını etiketleyip sertifikanın ekrandaki g&ouml;r&uuml;nt&uuml;s&uuml;n&uuml; ve kendiniz de g&ouml;r&uuml;lecek şekilde instagram hikayenizde paylaşabilirseniz seviniriz</strong>.&nbsp;<strong>Tabi isterseniz.</strong></p>
        <p style="font-weight: 400;">Sağlıklı g&uuml;nler dileriz,</p>
        <p style="font-weight: 400;">KUARteam</p>
        <p><a href="http://kuar.ku.edu.tr/" data-saferedirecturl="https://www.google.com/url?q=http://kuar.ku.edu.tr/&amp;source=gmail&amp;ust=1608311190421000&amp;usg=AFQjCNFijkCHoQK12hvXCa_st_dQ7Hx1ag">Ko&ccedil; University&nbsp;&ndash; Ar&ccedil;elik Research Center for Creative Industries</a></p>
        <p>+90 (212) 338 37 34</p>"""
        
        message = create_message_with_attachment('KUAR Research Center for Creative Industries', email, subject, replyto, message_text,fn)
        
        #send()
        

0 No Reply no-reply@ku.edu.tr certs\KUAR_certificate__Data Set 0.pdf


In [7]:
#send()